In [3]:
import pandas as pd

In [7]:
df = pd.read_excel("data/bdd_c.xlsx")

In [8]:
df

,cis,nom_commercial,dci,dosage,dosage_nature,atc,cip7
0,61266250,"A 313 200 000 UI POUR CENT, pommade","CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",200 000 UI,100 g de pommade,NaN,3000147
1,62869109,"A 313 50 000 U.I., capsule molle","CONCENTRAT DE VITAMINE A SYNTHÉTIQUE, FORME HU...",50 000 UI,une capsule,NaN,3000064
2,69103878,"A.D.N. BOIRON, degré de dilution compris entre...",A.D.N. POUR PRÉPARATIONS HOMÉOPATHIQUES,4CH à 30CH et 8DH à 60DH; 4CH à 30CH et 8DH à ...,un comprimé,NaN,NaN
3,61876780,"ABACAVIR ARROW 300 mg, comprimé pelliculé sécable",SULFATE D'ABACAVIR,NaN,NaN,J05AF06,3019221
4,63797011,"ABACAVIR SANDOZ 300 mg, comprimé pelliculé séc...",ABACAVIR,300 mg,un comprimé,J05AF06,3008995
...,...,...,...,...,...,...,...
15818,64949486,"ZYRTECSET 10 mg, comprimé pelliculé sécable",CÉTIRIZINE (DICHLORHYDRATE DE),10 mg,un comprimé,NaN,3646168
15819,67337081,"ZYTIGA 500 mg, comprimé pelliculé",ACÉTATE D'ABIRATÉRONE,500 mg,un comprimé,L02BX03,3007627
15820,63095061,"ZYVOXID 100 mg/5 ml, granulés pour suspension ...",LINÉZOLIDE,100 mg,5 ml de suspension reconstituée,J01XX08,5651268
15821,63283095,"ZYVOXID 2 mg/ml, solution pour perfusion",LINÉZOLIDE,2 mg,1 ml,J01XX08,5811074


In [10]:
import pandas as pd
import unidecode


df["dci"] = (
df["dci"]
    .astype(str)
    .apply(lambda x: unidecode.unidecode(x))  # enlever les accents
    .str.upper()                              # majuscules
    .str.strip()                              # trim
)


In [14]:
df_d1 = df.drop_duplicates(subset=['dci'], keep='first')

In [15]:
df_d1

,cis,nom_commercial,dci,dosage,dosage_nature,atc,cip7
0,61266250,"A 313 200 000 UI POUR CENT, pommade","CONCENTRAT DE VITAMINE A SYNTHETIQUE, FORME HU...",200 000 UI,100 g de pommade,NaN,3000147
2,69103878,"A.D.N. BOIRON, degré de dilution compris entre...",A.D.N. POUR PREPARATIONS HOMEOPATHIQUES,4CH à 30CH et 8DH à 60DH; 4CH à 30CH et 8DH à ...,un comprimé,NaN,NaN
3,61876780,"ABACAVIR ARROW 300 mg, comprimé pelliculé sécable",SULFATE D'ABACAVIR,NaN,NaN,J05AF06,3019221
4,63797011,"ABACAVIR SANDOZ 300 mg, comprimé pelliculé séc...",ABACAVIR,300 mg,un comprimé,J05AF06,3008995
6,68257528,"ABACAVIR/LAMIVUDINE ACCORD 600 mg/300 mg, comp...",CHLORHYDRATE D'ABACAVIR MONOHYDRATE; LAMIVUDINE,300 mg,un comprimé,NaN,3008342
...,...,...,...,...,...,...,...
15790,64556383,"ZYBAN L.P. 150 mg, comprimé à libération prolo...",CHLORHYDRATE DE BUPROPIONE,150 mg,un comprimé,NaN,3575149
15792,69111017,"ZYDELIG 100 mg, comprimé pelliculé",IDELALISIB,100 mg,un comprimé,L01EM01,2795966
15794,66958808,"ZYKADIA 150 mg, gélule",CERITINIB,150 mg,une gélule,L01ED02,3014611
15802,67512328,"ZYMADUO 150 UI, solution buvable en gouttes",CHOLECALCIFEROL; FLUORURE DE SODIUM,"0,1380 mg; 150 UI",une goutte,NaN,3478050


In [16]:
import re

def garder_premier_mot(m):
    return m.split(" ")[0].strip()


df["nom_commercial"] = df["nom_commercial"].apply(garder_premier_mot)


In [18]:
df_d = df.drop_duplicates(subset=['nom_commercial'], keep='first')

In [19]:
df_d

,cis,nom_commercial,dci,dosage,dosage_nature,atc,cip7
0,61266250,A,"CONCENTRAT DE VITAMINE A SYNTHETIQUE, FORME HU...",200 000 UI,100 g de pommade,NaN,3000147
2,69103878,A.D.N.,A.D.N. POUR PREPARATIONS HOMEOPATHIQUES,4CH à 30CH et 8DH à 60DH; 4CH à 30CH et 8DH à ...,un comprimé,NaN,NaN
3,61876780,ABACAVIR,SULFATE D'ABACAVIR,NaN,NaN,J05AF06,3019221
6,68257528,ABACAVIR/LAMIVUDINE,CHLORHYDRATE D'ABACAVIR MONOHYDRATE; LAMIVUDINE,300 mg,un comprimé,NaN,3008342
15,63152345,ABACAVIR/LAMIVUDINE/ZIDOVUDINE,LAMIVUDINE; SULFATE D'ABACAVIR; ZIDOVUDINE,150 mg; 300 mg,un comprimé,J05AR04,3012310
...,...,...,...,...,...,...,...
15810,60841335,ZYPREXA,OLANZAPINE,10 mg,un comprimé,N05AH03,3423801; 4950520; 4950655; 4950709; 4950951
15816,65016707,ZYRTEC,DICHLORHYDRATE DE CETIRIZINE,"1,000 g",100 ml,NaN,3417700
15817,62636293,ZYRTECSET,CETIRIZINE (DICHLORHYDRATE DE),10 mg,un comprimé,NaN,4900477
15819,67337081,ZYTIGA,ACETATE D'ABIRATERONE,500 mg,un comprimé,L02BX03,3007627


In [27]:
df_unique.to_excel("ess.xlsx")